## Installing and Importing libraries

In [ ]:
!pip install dash
!pip install jupyter-dash
!pip install dash_bootstrap_components

In [17]:
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
import dash_bootstrap_components as dbc
from dash import html
from dash.dependencies import Input, Output 
import pandas as pd
import numpy as np

# Data set

In [3]:
from google.colab import files
uploaded = files.upload()

Saving sample_sales_data.csv to sample_sales_data.csv


In [4]:
df = pd.read_csv("sample_sales_data.csv")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [5]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9789 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9789 non-null   int64  
 1   Order ID       9789 non-null   object 
 2   Order Date     9789 non-null   object 
 3   Ship Date      9789 non-null   object 
 4   Ship Mode      9789 non-null   object 
 5   Customer ID    9789 non-null   object 
 6   Customer Name  9789 non-null   object 
 7   Segment        9789 non-null   object 
 8   Country        9789 non-null   object 
 9   City           9789 non-null   object 
 10  State          9789 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9789 non-null   object 
 13  Product ID     9789 non-null   object 
 14  Category       9789 non-null   object 
 15  Sub-Category   9789 non-null   object 
 16  Product Name   9789 non-null   object 
 17  Sales          9789 non-null   float64
dtypes: float

# Data Manipulation and cleaning

In [6]:
df['Order_Year'] = pd.DatetimeIndex(df['Order Date']).year
df['Order_Month'] = pd.DatetimeIndex(df['Order Date']).month

In [7]:
df_new = df.groupby(['Region','Order_Year', 'Order_Month']).Sales.sum().reset_index()
df_new["Year_Month"] = pd.to_datetime(df_new['Order_Year'].astype(str) + '-' + df_new['Order_Month'].astype(str))
df_new.head()

,Region,Order_Year,Order_Month,Sales,Year_Month
0,Central,2015,1,10838.995,2015-01-01
1,Central,2015,2,2648.124,2015-02-01
2,Central,2015,3,5038.584,2015-03-01
3,Central,2015,4,2367.214,2015-04-01
4,Central,2015,5,4588.496,2015-05-01


# Dash app - 1

In [37]:
app = JupyterDash(external_stylesheets=[dbc.themes.DARKLY])

app.layout = html.Div(
    children = [
                html.H2(children = "Sales Data Across regions from 2015 to 2018",
                        style = {'color': 'white'}),
                dcc.Dropdown(id = "region-dropdown",
                             options = [{'label': i, 'value': i} for i in df_new["Region"].unique().tolist()],
                             value = "Central",
                             style = {'width': '20vw','display': 'inline-block','color': 'black', 'padding':'10px'}
                             ),
                dcc.Graph(id='sales-chart')          
    ]   
)

@app.callback(Output(component_id='sales-chart',
                     component_property='figure'),
              Input( component_id='region-dropdown',
                     component_property='value')
              )

def update_graph(region_name):
  final_df = df_new.groupby(["Region", "Order_Year", "Order_Month"]).Sales.sum().reset_index()
  final_df = final_df[final_df["Region"] == region_name]

  fig = px.line(final_df, x = "Order_Month", y = "Sales", color="Order_Year",
                title = f"Sales Trends for {region_name} Region",
                template = "plotly_dark")
  
  return fig

app.run_server(mode="external")
    


Dash app running on:


<IPython.core.display.Javascript object>

# Dash app - 2

In [8]:
new = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv")
df_temp = df.merge(new[['code', 'state']], left_on="State", right_on="state", how="left")

In [9]:
df_sales = df_temp.groupby(["code", "State", "Order_Year"]).Sales.sum().reset_index()
df_orders = df_temp.groupby(["code","State", "Order_Year"])["Order ID"].count().reset_index()
df_orders.columns = ["code",'State', 'Order_Year', 'Orders']

In [10]:
df_sales.head()

,code,State,Order_Year,Sales
0,AL,Alabama,2015,6139.09
1,AL,Alabama,2016,3891.97
2,AL,Alabama,2017,7651.33
3,AL,Alabama,2018,1828.25
4,AR,Arkansas,2015,6302.69


In [38]:
# from urllib.request import urlopen
# import json
# with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
#     counties = json.load(response)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash()

app.layout = html.Div([
    html.Div([
    html.H2(children = "State wise visualization",
            style = {'color': 'navy'}),
    html.Div(
    children = [
                html.Label("Sales or Orders:"),
                dcc.RadioItems(id = "radio-input",
                             options = [{'label': i, 'value': i} for i in ["Sales", "Orders"] ],
                             value = 'Sales',
                             style = {'width': '50vw','display': 'inline-block'}
                             ),           
    ], 
    style={'width': '30%', 'display': 'inline-block'}
    ),
   html.Div(
    children = [
                html.Label("Year:"),
                dcc.Dropdown(id = "year-input",
                             options = [{'label': i, 'value': i} for i in df_sales["Order_Year"].unique().tolist() ],
                             value = 2018,
                             style = {'width': '50vw','display': 'inline-block'}
                             ),           
    ],
    style={'width': '48%', 'display': 'inline-block'}
    ),
]),
    dcc.Graph(id='choropleth'),
    dcc.Graph(id='bar-graph')
])

@app.callback(Output(component_id='choropleth',
                     component_property='figure'),
              Input( component_id='radio-input',
                     component_property='value'),
              Input( component_id='year-input',
                     component_property='value'),
              )

def display_choropleth(radio_input, year_input):
  if radio_input == 'Sales':
    df_plot = df_sales[df_sales["Order_Year"] == year_input]
    col_scale = "Viridis"
  else:
    df_plot = df_orders[df_orders["Order_Year"] == year_input]
    col_scale = 'reds'
  title = f"{radio_input} across USA for the year {year_input}"
  fig = px.choropleth(df_plot, locations="code", color=f"{radio_input}", color_continuous_scale = col_scale, locationmode="USA-states", scope="usa",
                      title=title)
  # fig = px.choropleth(df_sales, locations="code", color="Sales", locationmode="USA-states", scope="usa")
  return fig

@app.callback(Output(component_id='bar-graph',
                     component_property='figure'),
              Input( component_id='radio-input',
                     component_property='value'),
              Input( component_id='year-input',
                     component_property='value'))

def update_graph(radio_input, year_input):
  if radio_input == 'Sales':
    df_plot = df_sales.groupby("State")["Sales"].sum().reset_index()
    col_scale = "blue"
  else:
    df_plot = df_orders.groupby("State")["Orders"].sum().reset_index()
    col_scale = 'red'
  fig = px.bar(df_plot, x = "State", y = f"{radio_input}",
                title = f"{radio_input} Trends for States for all years")
  return fig

app.run_server(mode="external")
    


Dash app running on:


<IPython.core.display.Javascript object>

# Dash app - 3 (tabs)

In [39]:
def region_dropdown():
    return html.Div([dcc.Dropdown(id = "region-dropdown",
                             options = [{'label': i, 'value': i} for i in df_new["Region"].unique().tolist()],
                             value = "Central",
                             style = {'width': '50vw','display': 'inline-block'}
                             ),]
                    ,className="dropdown")
    
def radiobuttons():
    return dcc.RadioItems(id = "radio-input",
                          options = [{'label': i, 'value': i} for i in ["Sales", "Orders"] ],
                          value = 'Sales',
                          style = {'width': '50vw','display': 'inline-block'}
                          )
def year_dropdown():
    return  dcc.Dropdown(id = "year-input",
                             options = [{'label': i, 'value': i} for i in df_sales["Order_Year"].unique().tolist() ],
                             value = 2018,
                             style = {'width': '50vw','display': 'inline-block'}
                             )

def sales_chart():
    return dcc.Graph(id="sales-chart")

def choropleth_chart():
    return dcc.Graph(id="choropleth")

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Sales Dashboard'),
    dcc.Tabs(id="tabs", value='tab-1', 
             children=[
                    dcc.Tab(label="Sales Trends by Region",
                            value="tab-1",
                            className="custom-tab",
                            children=[html.Div([html.H2(children = "Sales Data Across regions from 2015 to 2018",
                                                        style = {'color': 'navy'}),
                                                region_dropdown(),
                                                html.Br(),
                                                sales_chart()]),
                                      ]
                            ),
                       dcc.Tab(label="Sales/Orders by state and year",
                            value="tab-2",
                            className="custom-tab",
                            children=[html.Div([html.H2(children = "State wise visualization",
                                                        style = {'color': 'navy'}),
                                              radiobuttons(),    
                                              html.Label("Year:"),
                                              year_dropdown(),
                                            choropleth_chart()
                                      ]
                            ),
                ]),
      ])
])

def display_choropleth(radio_input, year_input):
  if radio_input == 'Sales':
    df_plot = df_sales[df_sales["Order_Year"] == year_input]
    col_scale = "Viridis"
  else:
    df_plot = df_orders[df_orders["Order_Year"] == year_input]
    col_scale = 'reds'

  # fig = px.bar(df_plot, x = "State", y = f"{radio_input}",
  #               title = f"{radio_input} Trends for States")
  fig = px.choropleth(df_plot, locations="code", color=f"{radio_input}", color_continuous_scale = col_scale, locationmode="USA-states", scope="usa")
  # fig = px.choropleth(df_sales, locations="code", color="Sales", locationmode="USA-states", scope="usa")
  return fig

@app.callback([Output("sales-chart", "figure"),
               Output("choropleth", "figure")
               ],
             [Input("tabs", "value"),
              Input(component_id='region-dropdown',
                     component_property='value'),
              Input( component_id='radio-input',
                     component_property='value'),
              Input( component_id='year-input',
                     component_property='value')])

def update_graph(tab, region_name, radio_input, year_input):
  if tab == 'tab-1':
      final_df = df_new.groupby(["Region", "Order_Year", "Order_Month"]).Sales.sum().reset_index()
      final_df = final_df[final_df["Region"] == region_name]

      fig = px.line(final_df, x = "Order_Month", y = "Sales", color="Order_Year",
                    title = f"Sales Trends for {region_name} Region")
      
      return fig
  else:
      display_choropleth(radio_input, year_input)


app.run_server(mode="external")

Dash app running on:


<IPython.core.display.Javascript object>